In [9]:
%cd /home/vagar/predict_expression

/home/vagar/predict_expression


In [ ]:
import os, h5py
import numpy as np
import numpy.random as npr
import pandas as pd
from sklearn import preprocessing
from sklearn.decomposition import PCA
from functools import reduce

maxshift = 10000
X = np.array([])

print("Round1")
for shift in list(range(-maxshift, maxshift + 1, 200)):
    print(shift)
    preds = h5py.File("embedded_vals/human_hg19_promoters_1nt_FantomCorrected.bed.shift_SHIFT.h5".replace(
        'SHIFT', str(shift)), 'r')['/pred']
    X = np.concatenate([X,preds]) if X.size else preds
    print(X.shape)

print("Fitting PCA")
pca = PCA(0.99, whiten=True) #Incremental, batch_size=10
X = pca.fit_transform(X)
print(X.shape)

maxshift = 100000
X = np.array([])
print("Round2")
for shift in list(range(-maxshift, maxshift + 1, 200)):
    print(shift)
    preds = h5py.File("embedded_vals/human_hg19_promoters_1nt_FantomCorrected.bed.shift_SHIFT.h5".replace(
        'SHIFT', str(shift)), 'r')['/pred']
    preds = pca.transform(preds)
    preds = np.expand_dims(preds,axis=1)
    X = np.hstack([X,preds]) if X.size else preds
    print(X.shape)

f = h5py.File('embedded_vals/X.h5', 'w')
f.create_dataset('pred', data=X)
f.close()

compress_args = {'compression': 'gzip', 'compression_opts': 1}
out_dir="prepared_data"
if not os.path.exists(out_dir):
    os.mkdir(out_dir)
outfile = os.path.join(out_dir, 'expr_preds.h5')

genes = pd.read_csv("human_hg19_promoters_1nt_FantomCorrected.bed",
                    sep="\t", header=None, names=['chr', 'start', 'end', '.', 'strand', '?', '??', '???', 'name'])
X = X[~genes.chr.isin(['chrY']),:,:]
genes = genes[~genes.chr.isin(['chrY'])] #'chrX',
y = pd.read_csv("57epigenomes.RPKM.pc", sep="\t", index_col=0)
maskedIDs = pd.read_table("mask_histone_genes.txt", header=None) #mask histone genes, chrY genes already filtered out
y = y[y.index.isin(genes['name']) & ~y.index.isin(maskedIDs[0])] #remove rows corresponding to chrY or histone sequences
X = X[genes['name'].isin(y.index),:,:]
genes = genes[genes['name'].isin(y.index)]
y = reduce(pd.DataFrame.append, map(lambda i: y[y.index == i], genes['name']))

# print X.shape, y.shape, genes.shape #np.swapaxes(X,1,2)
y = preprocessing.scale(np.log10(y+0.1))
idxs = np.arange(X.shape[0])
npr.seed(0)
npr.shuffle(idxs)

geneNames = np.array(genes[genes.columns[8]])[idxs]
X = X[idxs]
y = y[idxs]

# check that the sum is valid
test_count = 1000
valid_count = 1000
assert(test_count + valid_count <= X.shape[0])
train_count = X.shape[0] - test_count - valid_count

print('%d training sequences ' % train_count)
print('%d test sequences ' % test_count)
print('%d validation sequences ' % valid_count)
h5f = h5py.File(outfile, 'w')
i = 0
if train_count > 0:
    h5f.create_dataset('train_in'       , data=X[i:i+train_count,:], **compress_args)
    h5f.create_dataset('train_out'      , data=y[i:i+train_count], **compress_args)
    h5f.create_dataset('train_geneName' , data=geneNames[i:i+train_count].tolist(), **compress_args)
    h5f.close()
i += train_count
if valid_count > 0:
    h5f.create_dataset('valid_in'       , data=X[i:i+valid_count,:], **compress_args)
    h5f.create_dataset('valid_out'      , data=y[i:i+valid_count], **compress_args)
    h5f.create_dataset('valid_geneName' , data=geneNames[i:i+valid_count].tolist(), **compress_args)
    h5f.close()
i += valid_count
if test_count > 0:
    h5f.create_dataset('test_in'        , data=X[i:i+test_count,:], **compress_args)
    h5f.create_dataset('test_out'       , data=y[i:i+test_count], **compress_args)
    h5f.create_dataset('test_geneName'  , data=geneNames[i:i+test_count].tolist(), **compress_args)
    h5f.close()

Round1
-10000
(19071, 2002)
-9800
(38142, 2002)
-9600
(57213, 2002)
-9400
(76284, 2002)
-9200
(95355, 2002)
-9000
(114426, 2002)
-8800
(133497, 2002)
-8600
(152568, 2002)
-8400
(171639, 2002)
-8200
(190710, 2002)
-8000
(209781, 2002)
-7800
(228852, 2002)
-7600
(247923, 2002)
-7400
(266994, 2002)
-7200
(286065, 2002)
-7000
(305136, 2002)
-6800
(324207, 2002)
-6600
(343278, 2002)
-6400
(362349, 2002)
-6200
(381420, 2002)
-6000
(400491, 2002)
-5800
(419562, 2002)
-5600
(438633, 2002)
-5400
(457704, 2002)
-5200
(476775, 2002)
-5000
(495846, 2002)
-4800
(514917, 2002)
-4600
(533988, 2002)
-4400
(553059, 2002)
-4200
(572130, 2002)
-4000
(591201, 2002)
-3800
(610272, 2002)
-3600
(629343, 2002)
-3400
(648414, 2002)
-3200
(667485, 2002)
-3000
(686556, 2002)
-2800
(705627, 2002)
-2600
(724698, 2002)
-2400
(743769, 2002)
-2200
(762840, 2002)
-2000
(781911, 2002)
-1800
(800982, 2002)
-1600
(820053, 2002)
-1400
(839124, 2002)
-1200
(858195, 2002)
-1000
(877266, 2002)
-800
(896337, 2002)
-600
(91540